In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("./"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

We are tasked with creating an automated tetris player that can successfully play for an extended period of time. By using a Reinforcement Learning model we will achieve this goal with the several characteristics we can define and configure within the model to make it properly play the game.

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

The data is collected by real-time training through trial and error, the model observes the current state takes and action, then recieves either positive or negative feedback. Data is collected by completing several games overtime. 

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

The board consists of a 2D grid that is 10 x 20 that can take in binary values (0 = empty, 1 = filled). There are 7 types of blocks that will be taken into account along with the current roation state of those shapes. Some of the features that will be derived is the board height before each piece is placed, number of holes, bumpiness, number of complete lines, and the individual column heights. The model will be able to rotate the piece 0, 80, 180, or 270 degrees and move its horizontal position as long as it's within the confinments of the board. 

# 4.Prepare the Data


Apply any data transformations and explain what and why


To properly prepare the data we will need to flatten the 10x20 board to a 200-dimensional vector and keep the normslization of 0 for empty 1 for filled. Then feature engineering will include the hand-extracted features. We then need to scale the rewards to a resonable range where positive awards outweigh the negative awards to fuel efficiency in a Reinforcement Learning model. 

# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


I first attempted creating a DQN model however it came to my attention that not only was each generation very slow but it kept overfitting and learning mostly from failures and little to no successes so the results were very sparse and it never properly learned. I ended up using a genetic algorithm with heuristic weights which worked much better due to the immediate feedback and the heuristic weights were much more comparable to the factors that play into Tetris those being height, holes, bumpiness, etc... after each generation the model improves even slightly and trains much faster and efficiently.  

# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


In [ ]:
"""
Evolutionary Algorithm for Tetris
Evolves optimal weights for long-running games
Trains headless, runs with visual
"""
from copy import deepcopy
import numpy as np
import random
import os
import pickle

# Configuration
TRAINING_MODE = True  # False means run the best model saved
CHECKPOINT_DIR = "checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, "evolution.pkl")
LOG_FILE = os.path.join(CHECKPOINT_DIR, "evolution_log.txt")

POPULATION_SIZE = 10  
ELITE_SIZE = 4  
MUTATION_RATE = 0.4 
MUTATION_STRENGTH = 0.25  
GAMES_PER_INDIVIDUAL = 5  


def evaluate_board(board_array):
    """Calculate board features"""
    height = len(board_array)
    width = len(board_array[0])
    
    heights = []
    for col in range(width):
        col_height = 0
        for row in range(height):
            if board_array[row][col]:
                col_height = row + 1
        heights.append(col_height)
    
    aggregate_height = sum(heights)
    max_height = max(heights) if heights else 0
    
    holes = 0
    for col in range(width):
        found_block = False
        for row in range(height - 1, -1, -1):
            if board_array[row][col]:
                found_block = True
            elif found_block:
                holes += 1
    
    bumpiness = sum(abs(heights[i] - heights[i + 1]) for i in range(len(heights) - 1))
    complete_lines = sum(1 for row in board_array if all(row))
    
    wells = 0
    for i in range(width):
        if i == 0:
            if heights[i] < heights[i + 1]:
                wells += heights[i + 1] - heights[i]
        elif i == width - 1:
            if heights[i] < heights[i - 1]:
                wells += heights[i - 1] - heights[i]
        else:
            if heights[i] < heights[i - 1] and heights[i] < heights[i + 1]:
                wells += min(heights[i - 1], heights[i + 1]) - heights[i]
    
    return {
        'aggregate_height': aggregate_height,
        'max_height': max_height,
        'holes': holes,
        'bumpiness': bumpiness,
        'complete_lines': complete_lines,
        'wells': wells
    }

def simulate_placement(board, piece, x):
    """Simulate placing a piece"""
    board_copy = [row[:] for row in board.board[:board.height]]
    
    try:
        y = board.drop_height(piece, x)
    except:
        return None, 0
    
    for pos in piece.body:
        row_idx = y + pos[1]
        col_idx = x + pos[0]
        if 0 <= row_idx < board.height and 0 <= col_idx < board.width:
            board_copy[row_idx][col_idx] = True
    
    lines_cleared = sum(1 for row in board_copy if all(row))
    
    return board_copy, lines_cleared

#Agent
class EvolutionaryAgent:
    def __init__(self):
        # Start with current best weights
        self.best_weights = {
            'aggregate_height': -0.8385,
            'max_height': -1.0613,
            'holes': -1.1523,
            'bumpiness': -0.0481,
            'complete_lines': 0.4979,
            'wells': -0.5756
        }
        
        self.population = []
        self.generation = 0
        self.best_fitness = 0
        self.best_pieces = 0
        self.best_rows = 0
        self.evaluations = 0
        
        # Try to load checkpoint
        if os.path.exists(CHECKPOINT_PATH):
            self.load_checkpoint()
            print(f"Loaded evolution: Gen {self.generation}, Best fitness: {self.best_fitness:.0f}")
            print(f"Best performance: {self.best_pieces} pieces, {self.best_rows} rows")
        else:
            print("Starting evolution from scratch")
            self.initialize_population()
    
    def initialize_population(self):
        """Create initial population around best known weights"""
        self.population = []
        
        # Add the best known weights
        self.population.append({
            'weights': self.best_weights.copy(),
            'fitness': 0,
            'games_played': 0
        })
        
        # Create variations
        for _ in range(POPULATION_SIZE - 1):
            weights = {}
            for key, value in self.best_weights.items():
                # Add random variation
                noise = random.gauss(0, 0.3)
                weights[key] = value + noise
            
            self.population.append({
                'weights': weights,
                'fitness': 0,
                'games_played': 0
            })
    
    def evaluate_position(self, board, piece, x, weights):
        """Evaluate a position using given weights"""
        simulated_board, lines_cleared = simulate_placement(board, piece, x)
        
        if simulated_board is None:
            return -999999
        
        features = evaluate_board(simulated_board)
        
        score = 0
        for feature, value in features.items():
            score += weights[feature] * value
        
        return score
    
    def get_best_move(self, board, piece, weights=None):
        """Get best move using given weights"""
        if weights is None:
            weights = self.best_weights
        
        best_x = -1
        best_piece = None
        best_score = -999999
        
        current_piece = piece
        for rotation in range(4):
            for x in range(board.width):
                if x + len(current_piece.skirt) > board.width:
                    continue
                
                try:
                    score = self.evaluate_position(board, current_piece, x, weights)
                    if score > best_score:
                        best_score = score
                        best_x = x
                        best_piece = current_piece
                except:
                    pass
            
            current_piece = current_piece.get_next_rotation()
        
        if best_x == -1:
            best_x = 0
            best_piece = piece
        
        return best_x, best_piece
    
    def evolve_generation(self):
        """Evolve to next generation"""
        # Sort by fitness
        self.population.sort(key=lambda x: x['fitness'], reverse=True)
        
        # Update best
        if self.population[0]['fitness'] > self.best_fitness:
            self.best_fitness = self.population[0]['fitness']
            self.best_weights = self.population[0]['weights'].copy()
        
        # Keep elite
        new_population = self.population[:ELITE_SIZE]
        
        # Create offspring
        while len(new_population) < POPULATION_SIZE:
            # Select parents from elite
            parent1 = random.choice(self.population[:ELITE_SIZE])
            parent2 = random.choice(self.population[:ELITE_SIZE])
            
            # Crossover
            child_weights = {}
            for key in self.best_weights.keys():
                if random.random() < 0.5:
                    child_weights[key] = parent1['weights'][key]
                else:
                    child_weights[key] = parent2['weights'][key]
                
                # Mutation
                if random.random() < MUTATION_RATE:
                    noise = random.gauss(0, MUTATION_STRENGTH)
                    child_weights[key] += noise
            
            new_population.append({
                'weights': child_weights,
                'fitness': 0,
                'games_played': 0
            })
        
        self.population = new_population
        self.generation += 1
        
        # Save checkpoint
        self.save_checkpoint()
    
    def save_checkpoint(self):
        """Save evolution state"""
        data = {
            'best_weights': self.best_weights,
            'population': self.population,
            'generation': self.generation,
            'best_fitness': self.best_fitness,
            'best_pieces': self.best_pieces,
            'best_rows': self.best_rows,
            'evaluations': self.evaluations
        }
        with open(CHECKPOINT_PATH, 'wb') as f:
            pickle.dump(data, f)
    
    def load_checkpoint(self):
        """Load evolution state"""
        with open(CHECKPOINT_PATH, 'rb') as f:
            data = pickle.load(f)
        self.best_weights = data['best_weights']
        self.population = data.get('population', [])
        self.generation = data.get('generation', 0)
        self.best_fitness = data.get('best_fitness', 0)
        self.best_pieces = data.get('best_pieces', 0)
        self.best_rows = data.get('best_rows', 0)
        self.evaluations = data.get('evaluations', 0)
        
        if not self.population:
            self.initialize_population()

# ============================================================================
# MAIN AI CLASS
# ============================================================================
_agent = None
_first_game = True
_current_individual = 0
_current_game_in_eval = 0

class CUSTOM_AI_MODEL:
    def __init__(self):
        global _agent, _first_game
        
        if _agent is None:
            _agent = EvolutionaryAgent()
        
        self.agent = _agent
        self.pieces_count = 0
        self.rows_count = 0
        self.game_over = False
        
        if TRAINING_MODE and self.agent.population:
            self.current_weights = self.agent.population[_current_individual]['weights']
        else:
            self.current_weights = self.agent.best_weights
        
        if _first_game:
            _first_game = False
            if TRAINING_MODE:
                print("\n" + "="*80)
                print("EVOLUTIONARY TETRIS AI - TRAINING MODE")
                print("="*80)
                print(f"Generation: {self.agent.generation}")
                print(f"Population size: {POPULATION_SIZE}")
                print(f"Games per individual: {GAMES_PER_INDIVIDUAL}")
                print(f"Games per generation: {POPULATION_SIZE * GAMES_PER_INDIVIDUAL}")
                print(f"Best fitness so far: {self.agent.best_fitness:.0f}")
                print(f"Best performance: {self.agent.best_pieces}p/{self.agent.best_rows}r")
                print(f"Checkpoint: {CHECKPOINT_PATH}")
                print(f"Log file: {LOG_FILE}")
                print("="*80 + "\n")
                
                # Write initial log entry
                with open(LOG_FILE, 'a') as f:
                    f.write(f"\n{'='*80}\n")
                    f.write(f"Training resumed at generation {self.agent.generation}\n")
                    f.write(f"Best so far: {self.agent.best_pieces}p/{self.agent.best_rows}r\n")
                    f.write(f"{'='*80}\n\n")
                    f.flush()
            else:
                print("\n" + "="*80)
                print("EVOLUTIONARY TETRIS AI - PLAY MODE")
                print("="*80)
                print("Using best evolved weights:")
                for key, value in self.agent.best_weights.items():
                    print(f"  {key}: {value:.3f}")
                print("="*80 + "\n")
    
    def get_best_move(self, board, piece, depth=1):
        """Main method called by game.py"""
        x, selected_piece = self.agent.get_best_move(board, piece, self.current_weights)
        self.pieces_count += 1
        return x, selected_piece
    
    def __del__(self):
        """Called when game ends - handle evolution"""
        if not hasattr(self, 'pieces_count') or not TRAINING_MODE:
            return
        
        if self.pieces_count > 0 and not self.game_over:
            self.game_over = True
            self._continue_evolution()
    
    def _continue_evolution(self):
        """Continue evolutionary training"""
        global _current_individual, _current_game_in_eval
        from game import Game
        
        try:
            while True:
                # Run a game
                g = Game("student")
                pieces, rows = g.run_no_visual()
                
                # Evaluate current individual
                individual = self.agent.population[_current_individual]
                
                # Update fitness - HEAVILY PRIORITIZE ROWS WITH HARSH PENALTIES
                fitness = pieces + rows * 100  # Rows worth 100 pieces
                
                # HARSH PENALTIES for poor performance
                if pieces < 1000:
                    fitness *= 0.5  # 50% penalty for dying too early
                if rows < 100:
                    fitness *= 0.7  # 30% penalty for not clearing enough rows
                if rows == 0:
                    fitness *= 0.3  # 70% penalty for clearing NO rows
                
                # BONUSES for excellent performance
                if pieces > 50000:
                    fitness *= 1.5  # 50% bonus for very long games
                if rows > 20000:
                    fitness *= 2.0  # 100% bonus for clearing tons of rows
                
                individual['fitness'] += fitness
                individual['games_played'] += 1
                
                _current_game_in_eval += 1
                self.agent.evaluations += 1
                
                # Track best and save immediately when improved
                if pieces > self.agent.best_pieces:
                    self.agent.best_pieces = pieces
                    print(f"*** NEW BEST PIECES: {pieces} ***")
                    self.agent.save_checkpoint()
                    
                if rows > self.agent.best_rows:
                    self.agent.best_rows = rows
                    print(f"*** NEW BEST ROWS: {rows} ***")
                    self.agent.save_checkpoint()
                
                # Also save if overall fitness is best
                current_fitness = pieces + rows * 100
                if current_fitness > self.agent.best_fitness:
                    print(f"*** NEW BEST FITNESS: {current_fitness:.0f} ***")
                    self.agent.best_fitness = current_fitness
                    self.agent.save_checkpoint()
                
                print(f"Gen {self.agent.generation} | "
                      f"Ind {_current_individual + 1}/{POPULATION_SIZE} | "
                      f"Game {_current_game_in_eval}/{GAMES_PER_INDIVIDUAL} | "
                      f"Pieces: {pieces:5d} | Rows: {rows:5d} | "
                      f"Fitness: {fitness:7.0f}")
                
                # Move to next individual or generation
                if _current_game_in_eval >= GAMES_PER_INDIVIDUAL:
                    individual['fitness'] /= GAMES_PER_INDIVIDUAL
                    
                    _current_game_in_eval = 0
                    _current_individual += 1
                    
                    if _current_individual >= POPULATION_SIZE:
                        # Generation complete
                        _current_individual = 0
                        
                        avg_fitness = sum(ind['fitness'] for ind in self.agent.population) / POPULATION_SIZE
                        best_ind = max(self.agent.population, key=lambda x: x['fitness'])
                        
                        log_msg = (f"\n{'='*80}\n"
                                 f"GENERATION {self.agent.generation} COMPLETE\n"
                                 f"{'='*80}\n"
                                 f"Average fitness: {avg_fitness:.0f}\n"
                                 f"Best fitness this gen: {best_ind['fitness']:.0f}\n"
                                 f"All-time best fitness: {self.agent.best_fitness:.0f}\n"
                                 f"All-time best pieces: {self.agent.best_pieces:,}\n"
                                 f"All-time best rows: {self.agent.best_rows:,}\n"
                                 f"Total evaluations: {self.agent.evaluations}\n"
                                 f"{'='*80}\n")
                        print(log_msg)
                        
                        # Write to log file with flush
                        with open(LOG_FILE, 'a') as f:
                            f.write(log_msg)
                            f.flush()
                        
                        # Evolve to next generation
                        self.agent.evolve_generation()
                        
                        # Save after evolution
                        self.agent.save_checkpoint()
                        
                        print(f"Evolved to generation {self.agent.generation}")
                        print(f"Checkpoint saved: {CHECKPOINT_PATH}")
                        print(f"Log saved: {LOG_FILE}\n")
                
        except KeyboardInterrupt:
            print("\n" + "="*80)
            print("Evolution stopped by user")
            print(f"Generation: {self.agent.generation}")
            print(f"Best fitness: {self.agent.best_fitness:.0f}")
            print(f"Best performance: {self.agent.best_pieces}p/{self.agent.best_rows}r")
            print("="*80)
            self.agent.save_checkpoint()
            import sys
            sys.exit(0)



# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


The model employs a genetic algorithm to evolve certain heuristic weights for a board evaluation function. For each piece placement decision, the algorithm simulates all possible positions and rotations in a game of Tetris, it then evaluates each resulting board state using six weighted features (aggregate height, maximum height, holes, bumpiness, complete lines, and wells), and selects the placement with the highest weighted score. The weights are evolved over generations using a population of 20 individuals, where each individual represents a unique weight tested across 5 games to compute average fitness. After evaluation, the top 4 individuals are preserved, while the remaining population is generated through crossover and mutation of the elite parents. This process iterates across generations, with the target network updating after each generation to incorporate the best-performing weights.

# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 
